In [ ]:
# import tools
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
from scipy.fft import fft, fftfreq
import scipy
import plotly.graph_objects as go


In [ ]:
# Define some useful constants

# sample rate of the measurements
sampleRate = 1.8e6

# things for filtering
nyq = .5 * sampleRate
ctrFreq = 137.620e6


In [ ]:
# import the binary files
rawData = np.fromfile(
    "/Users/benjaminpattison/Documents/Projects/satNav/SDR_satellite_tracking/data/NOAA_15_pass.raw",
    dtype="uint32",
)
print(rawData[0:9])
np.size(rawData)/2


In [ ]:
start = int(75 * 1.8e6)
stop = int(76 * 1.8e6)
data = rawData[start:stop]
print(data[0:9])
np.size(data)/sampleRate

In [ ]:
# subtract 4294967295/2 to go from unsigned to signed values
data = data - (4294967295/2)

# split the data into its real and imaginary parts
RealParts = data[0::2]
ImagParts = data[1::2]

# subtract the mean to lose the DC offset
RealParts = RealParts - np.mean(RealParts)
ImagParts = ImagParts - np.mean(ImagParts)

# create a variable that has the real and imaginary parts together
cplx = RealParts + 1j * ImagParts

In [ ]:
cplx[:10]
plt.psd(cplx, NFFT=4096, Fs=sampleRate, Fc=ctrFreq)
plt.xlabel("Freq [Hz]")
plt.ylabel("Magnitude")
plt.title("PSD")
plt.show()

plt.plot(RealParts)
plt.xlabel("Sample")
plt.ylabel("Magnitude")
plt.title("Reals")
plt.show()

plt.plot(ImagParts)
plt.xlabel("Sample")
plt.ylabel("Magnitude")
plt.title("Imag")
plt.show()

In [ ]:
# Apply a filter to the reference signals

# https://scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html
sos = signal.butter(10, .02e6, btype="low", fs=sampleRate, output="sos")

# the ones in campbell first
sigfilt = signal.sosfilt(sos, cplx)

w, h = signal.sosfreqz(sos,fs=sampleRate)
plt.plot(w, np.abs(h))
plt.xlabel("Frequency [Hz]")
plt.ylabel("Frequency response")
plt.title("Low pass filter for signals")
plt.show()

In [ ]:
w, h = signal.freqz(cplx, fs=sampleRate)


In [ ]:
plt.plot(w, np.abs(h))
plt.xlabel("Frequency [Hz]")
plt.ylabel("Frequency response")
plt.title("Unfiltered response from campbell")
plt.show()




In [ ]:
# plot the frequency response of the reference and then the filtered reference
fig, axs = plt.subplots(2,1)
n = len(cplx)
yf = fft(cplx) * 1e-6
xf = fftfreq(n, 1/sampleRate) + ctrFreq
axs[0].plot(xf,np.abs(yf))
axs[0].set_title("Raw signal")

n = len(sigfilt)
yf = fft(sigfilt) * 1e-6
xf = fftfreq(n, 1/sampleRate) + ctrFreq
axs[1].plot(xf,np.abs(yf))
axs[1].set_title("Filtered")
axs[1].set_xlim([ctrFreq-20e3,ctrFreq+20e3])

for ax in axs.flat:
    ax.set(xlabel='Frequency [Hz]', ylabel='y-label')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()
    

In [ ]:
# plot the spectrograms of the filtered reference signals

pxx, freqs, bins, im = plt.specgram(cplx, NFFT=16*1024, Fs=sampleRate, noverlap=0, scale="dB")
plt.xlabel("Time [seconds]")
plt.ylabel("Frequency (deviation from 98.5 MHz) [Hz]")
plt.title("Campbell ground station 1st reference sample frequency response")
plt.show()

pxx, freqs, bins, im = plt.specgram(sigfilt, NFFT=16*1024, Fs=sampleRate, noverlap=0, scale="dB")
plt.xlabel("Time [seconds]")
plt.ylabel("Frequency (deviation from 98.5 MHz) [Hz]")
plt.title("Cupertino ground station 1st reference sample frequency response")
plt.show()


In [ ]:
plt.plot(xf,yf)
plt.show()

In [ ]:
# Correlation of the two reference signals using the differential phase method discussed here: http://www.panoradio-sdr.de/correlation-for-time-delay-analysis/
# and referencing some of the code from the matlab scripts in his library
"""
Transmitter location according to wikipedia
https://geohack.toolforge.org/geohack.php?pagename=KUFX&params=37.205_N_121.950_W_type:landmark_region:US-CA_source:FCC
repeaters here
    37.151583    -121.609917 (SE of SJ)
    37.659389    -121.933028 (NE of Fremont)
"""
# The first reference
# diff the signal
dPhase1_benGS = np.diff(np.unwrap(np.angle(benRef1_filtered)))
dPhase1_domGS = np.diff(np.unwrap(np.angle(domRef1_filtered)))

# remove the mean
dPhase1_benGS = dPhase1_benGS - np.mean(dPhase1_benGS)
dPhase1_domGS = dPhase1_domGS - np.mean(dPhase1_domGS)

# correlate the signals
dPhaseXcorr1 = signal.correlate(dPhase1_benGS, dPhase1_domGS)
dPhaseXcorr1_lags = signal.correlation_lags(len(dPhase1_benGS), len(dPhase1_domGS))
dPhaseXcorr1_max = np.max(dPhaseXcorr1)

refSig1Lag_samples = np.abs(dPhaseXcorr1_lags[np.argmax(dPhaseXcorr1)])
refSig1Lag_time = refSig1Lag_samples / sampleRate

# same process, for the second reference signal
dPhase2_benGS = np.diff(np.unwrap(np.angle(benRef2_filtered)))
dPhase2_domGS = np.diff(np.unwrap(np.angle(domRef2_filtered)))

dPhase_benGS = dPhase2_benGS - np.mean(dPhase2_benGS)
dPhase_domGS = dPhase2_domGS - np.mean(dPhase2_domGS)

dPhaseXcorr2 = signal.correlate(dPhase2_benGS, dPhase2_domGS)
dPhaseXcorr2_lags = signal.correlation_lags(len(dPhase2_benGS), len(dPhase2_domGS))
dPhaseXcorr_max2 = np.max(dPhaseXcorr1)

refSig2Lag_samples = np.abs(dPhaseXcorr2_lags[np.argmax(dPhaseXcorr2)])
refSig2Lag_time = refSig2Lag_samples / sampleRate

# print out the two lags (in units of seconds)
print(
    f"For the first reference: the lag in time is {refSig1Lag_time} and the lag in samples is {refSig1Lag_samples}"
)
print(
    f"For the second reference: the lag in time is {refSig2Lag_time} and the lag in samples is {refSig2Lag_samples}"
)


In [ ]:
# plot some things
# plot the original spectrum

start = int(2e6)
stop = start + 500

plt.subplot(2,1,1)
plt.plot(t[start:stop], benCplx[start:stop])
plt.plot(t[start:stop], domCplx[start:stop])
plt.xlabel("Time [seconds]")
plt.ylabel("Signal [units tbd]")
plt.title("Original Complex Signals (real parts only)")

plt.subplot(2,1,2)
plt.plot(t[start:stop], np.angle(benCplx[start:stop]))
plt.plot(t[start:stop], np.angle(domCplx[start:stop]))
plt.xlabel("Time [seconds]")
plt.ylabel("Phase angle [radians probably]")

plt.show()

In [ ]:
# plot some things
# plot the differential phase

start = int(2e6)
stop = start + 100


plt.plot(t[start:stop], dPhase1_benGS[start:stop])
plt.plot(t[start:stop], dPhase1_domGS[start:stop])
plt.xlabel("Time [seconds]")
plt.ylabel("Phase angle [radians probably]")
plt.title("Differential phase")
plt.show()

In [ ]:
# plot some things
# plot the spectrogram (PSD) of the references for each ground station
# note that frequency=0 corresponds to the center frequency, in this case 98.5 MHz

pxx, freqs, bins, im = plt.specgram(benCplx[: nS_each - gate], NFFT=16*1024, Fs=sampleRate, noverlap=0, scale="dB")
plt.ylim([-250000,250000])
plt.xlabel("Time [seconds]")
plt.ylabel("Frequency (deviation from 98.5 MHz) [Hz]")
plt.title("Campbell ground station 1st reference sample frequency response")
plt.show()

pxx, freqs, bins, im = plt.specgram(domCplx[: nS_each - gate], NFFT=16*1024, Fs=sampleRate, noverlap=0, scale="dB")
plt.ylim([-250000,250000])
plt.xlabel("Time [seconds]")
plt.ylabel("Frequency (deviation from 98.5 MHz) [Hz]")
plt.title("Cupertino ground station 1st reference sample frequency response")
plt.show()

pxx, freqs, bins, im = plt.specgram(benCplx[-nS_each + gate :], NFFT=16*1024, Fs=sampleRate, noverlap=0, scale="dB")
plt.ylim([-250000,250000])
plt.xlabel("Time [seconds]")
plt.ylabel("Frequency (deviation from 98.5 MHz) [Hz]")
plt.title("Campbell ground station 2nd reference sample frequency response")
plt.show()

pxx, freqs, bins, im = plt.specgram(domCplx[-nS_each + gate :], NFFT=16*1024, Fs=sampleRate, noverlap=0, scale="dB")
plt.ylim([-250000,250000])
plt.xlabel("Time [seconds]")
plt.ylabel("Frequency (deviation from 98.5 MHz) [Hz]")
plt.title("Cupertino ground station 2nd reference sample frequency response")
plt.show()


In [ ]:
# plot some things
# plot the spectrogram (PSD) of the references for each ground station
# note that frequency=0 corresponds to the center frequency, in this case 98.5 MHz

pxx, freqs, bins, im = plt.specgram(benCplx[: nS_each - gate], NFFT=16*1024, Fs=sampleRate, noverlap=0, mode="phase")
plt.ylim([-np.pi,np.pi])
plt.xlabel("Time [seconds]")
plt.ylabel("Phase angle [radians]")
plt.title("Campbell ground station 1st reference sample frequency response")
plt.show()

pxx, freqs, bins, im = plt.specgram(domCplx[: nS_each - gate], NFFT=16*1024, Fs=sampleRate, noverlap=0, mode="phase")
plt.ylim([-np.pi,np.pi])
plt.xlabel("Time [seconds]")
plt.ylabel("Phase angle [radians]")
plt.title("Cupertino ground station 1st reference sample frequency response")
plt.show()

pxx, freqs, bins, im = plt.specgram(benCplx[-nS_each + gate :], NFFT=16*1024, Fs=sampleRate, noverlap=0, mode="phase")
plt.ylim([-np.pi,np.pi])
plt.xlabel("Time [seconds]")
plt.ylabel("Phase angle [radians]")
plt.title("Campbell ground station 2nd reference sample frequency response")
plt.show()

pxx, freqs, bins, im = plt.specgram(domCplx[-nS_each + gate :], NFFT=16*1024, Fs=sampleRate, noverlap=0, mode="phase")
plt.ylim([-np.pi,np.pi])
plt.xlabel("Time [seconds]")
plt.ylabel("Phase angle [radians]")
plt.title("Cupertino ground station 2nd reference sample frequency response")
plt.show()

In [ ]:
nyq = .5 * sampleRate
bandPass = [98300000 / nyq, 98800000 / nyq]
sos = signal.butter(10, bandPass, btype="bandpass", fs=sampleRate, output="sos")
benCplx_filtered = signal.sosfilt(sos, benCplx)

start = int(2e6)
stop = start + 500

plt.plot(t[start:stop], benCplx[start:stop])
plt.xlabel("Time [seconds]")
plt.ylabel("Signal [units tbd]")
plt.title("Original Complex Signals (real parts only)")
plt.show()

plt.plot(t[start:stop], benCplx_filtered[start:stop])
plt.xlabel("Time [seconds]")
plt.ylabel("Signal [units tbd]")
plt.title("Filtered Complex Signals (real parts only)")
plt.show()

pxx, freqs, bins, im = plt.specgram(benCplx[gate : nS_each - gate], NFFT=16*1024, Fs=sampleRate, noverlap=0, scale="dB")
plt.ylim([-250000,250000])
plt.xlabel("Time [seconds]")
plt.ylabel("Frequency (deviation from 98.5 MHz) [Hz]")
plt.title("Campbell ground station 1st reference sample frequency response")
plt.show()

pxx, freqs, bins, im = plt.specgram(benCplx_filtered[gate : nS_each - gate], NFFT=16*1024, Fs=sampleRate, noverlap=0, scale="dB")
plt.ylim([-250000,250000])
plt.xlabel("Time [seconds]")
plt.ylabel("Frequency (deviation from 98.5 MHz) [Hz]")
plt.title("Campbell ground station 1st reference sample frequency response")
plt.show()

In [ ]:
sampleRate/2